In [1]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import threading

# Directory containing known faces
KNOWN_FACES_DIR = r"C:\attendance2\images"
TOLERANCE = 0.6  # Recognition tolerance
FRAME_THICKNESS = 3  # Thickness of bounding box
FONT_THICKNESS = 2  # Thickness of text
MODEL = "cnn"  # The model can be "hog" or "cnn" for detection

# Load known faces and their encodings
def load_known_faces():
    known_faces = []
    known_names = []

    for name in os.listdir(KNOWN_FACES_DIR):
        for filename in os.listdir(f"{KNOWN_FACES_DIR}/{name}"):
            image = face_recognition.load_image_file(f"{KNOWN_FACES_DIR}/{name}/{filename}")
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
    print("Loaded known faces")
    return known_faces, known_names

# Draw bounding boxes and names
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        # Draw a rectangle around the face
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)

        # Draw a label with a name below the face
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognize faces from live video
def recognize_faces_from_camera(camera_id, known_faces, known_names, tracking_dict):
    cap = cv2.VideoCapture(camera_id)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces with RetinaFace
        faces = RetinaFace.detect_faces(frame)
        print(f"Frame recorded from Camera {camera_id}")

        # Prepare face encodings for recognition
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []

        current_faces = set()  # To track currently detected faces

        for face_key in faces.keys():
            facial_area = faces[face_key]["facial_area"]

            # Append detected face locations for recognition
            face_locations.append(facial_area)
            
            current_faces.add(facial_area)
            
            # Convert RetinaFace coordinates to dlib format
            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            # Encode the face using face_recognition
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

            for encoding in encodings:
                # Compare with known faces
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"

                # Find the shortest distance to a known face
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]
                face_names.append(name)

                if name not in tracking_dict['seen']:
                    if camera_id == 0:
                        # Log entry time when a person is first detected by Camera 0
                        tracking_dict['entry_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['seen'][name] = datetime.now()
                        tracking_dict['exit_seen'][name] = False
                    elif camera_id == 1:
                        # Log exit time when a person is first detected by Camera 1
                        if not tracking_dict['exit_seen'].get(name, False):
                            tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                            tracking_dict['exit_seen'][name] = True

        # Detect absent persons and log exit times if Camera 0
        if camera_id == 0:
            absent_names = set(tracking_dict['seen'].keys()) - set(face_names)
            for name in absent_names:
                if not tracking_dict['exit_seen'][name]:
                    if datetime.now() - tracking_dict['seen'][name] >= timedelta(minutes=2):
                        tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        tracking_dict['exit_seen'][name] = True

        # Draw the results
        draw_faces(frame, face_locations, face_names)

        # Display the frame
        cv2.imshow(f'Camera {camera_id}', frame)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    # Load known faces
    known_faces, known_names = load_known_faces()
    print("Loaded Done")
    print(known_names)

    # Open CSV files for logging
    with open(r'C:\attendance2\attendance_log.csv', mode='a', newline='') as file:
        entry_writer = csv.writer(file)
        exit_writer = csv.writer(file)

        tracking_dict = {
            'entry_writer': entry_writer,
            'exit_writer': exit_writer,
            'seen': {},  # To track when a person was last seen
            'exit_seen': {}  # To track if exit has been recorded for the person
        }

        # Start tracking with two cameras
        camera_ids = [0, 1]  # Adjust these based on your camera IDs
        threads = []

        for camera_id in camera_ids:
            thread = threading.Thread(target=recognize_faces_from_camera, args=(camera_id, known_faces, known_names, tracking_dict))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

if __name__ == "__main__":
    main()



Loaded known faces
Loaded Done
['akhil', 'sadhvika', 'shreyesh', 'shreyesh', 'srinath']


Exception in thread Thread-5 (recognize_faces_from_camera):
Traceback (most recent call last):
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\gshreyeshreddy\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gshreyeshreddy\AppData\Local\Temp\ipykernel_27584\20586387.py", line 69, in recognize_faces_from_camera
TypeError: unhashable type: 'list'


Frame recorded from Camera 0
Frame recorded from Camera 1


Exception in thread Thread-6 (recognize_faces_from_camera):
Traceback (most recent call last):
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\gshreyeshreddy\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gshreyeshreddy\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gshreyeshreddy\AppData\Local\Temp\ipykernel_27584\20586387.py", line 69, in recognize_faces_from_camera
TypeError: unhashable type: 'list'


Frame recorded from Camera 1


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import threading

KNOWN_FACES_DIR = r"C:\attendance2\images"
CSV_FILE_PATH = r"C:\attendance2\attendance_log.csv"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"  # Can be "hog" for CPU-friendly detection
lock = threading.Lock()  # Lock to handle concurrency in file writing

def load_known_faces():
    known_faces = []
    known_names = []

    print("Loading known faces...")
    for name in os.listdir(KNOWN_FACES_DIR):
        person_dir = os.path.join(KNOWN_FACES_DIR, name)
        for filename in os.listdir(person_dir):
            filepath = os.path.join(person_dir, filename)
            image = face_recognition.load_image_file(filepath)
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
            else:
                print(f"Warning: No face found in {filename}")
    print("Finished loading known faces.")
    return known_faces, known_names

def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

def recognize_faces_from_camera(camera_id, known_faces, known_names, tracking_dict):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Camera {camera_id} failed to open.")
        return

    print(f"Camera {camera_id} started.")
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print(f"Failed to read frame from Camera {camera_id}.")
            break

        faces = RetinaFace.detect_faces(frame)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []

        for key in faces.keys():
            facial_area = faces[key]["facial_area"]
            face_locations.append(facial_area)

            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

            for encoding in encodings:
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)

                if matches[best_match_index]:
                    name = known_names[best_match_index]
                face_names.append(name)

                print(f"Detected {name} on Camera {camera_id}.")
                with lock:
                    if name not in tracking_dict['seen']:
                        if camera_id == 0:
                            tracking_dict['entry_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                            print(f"Entry logged for {name}.")
                            tracking_dict['seen'][name] = datetime.now()
                            tracking_dict['exit_seen'][name] = False
                            tracking_dict['file'].flush()  # Ensure immediate write
                        elif camera_id == 1 and not tracking_dict['exit_seen'].get(name, False):
                            tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                            print(f"Exit logged for {name}.")
                            tracking_dict['exit_seen'][name] = True
                            tracking_dict['file'].flush()

        if camera_id == 0:
            absent_names = set(tracking_dict['seen'].keys()) - set(face_names)
            for name in absent_names:
                if not tracking_dict['exit_seen'][name] and datetime.now() - tracking_dict['seen'][name] >= timedelta(minutes=2):
                    with lock:
                        tracking_dict['exit_writer'].writerow([name, datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
                        print(f"Auto exit logged for {name}.")
                        tracking_dict['exit_seen'][name] = True
                        tracking_dict['file'].flush()

        draw_faces(frame, face_locations, face_names)
        cv2.imshow(f'Camera {camera_id}', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print(f"Camera {camera_id} stopped.")
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    print("Starting attendance system...")
    known_faces, known_names = load_known_faces()

    with open(CSV_FILE_PATH, mode='a', newline='') as file:
        entry_writer = csv.writer(file)
        exit_writer = csv.writer(file)

        tracking_dict = {
            'entry_writer': entry_writer,
            'exit_writer': exit_writer,
            'seen': {},
            'exit_seen': {},
            'file': file  # Keep reference to the file for flushing
        }

        camera_ids = [0, 1]
        threads = []

        for camera_id in camera_ids:
            print(f"Starting thread for Camera {camera_id}...")
            thread = threading.Thread(target=recognize_faces_from_camera, args=(camera_id, known_faces, known_names, tracking_dict))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

    print("Attendance system stopped.")

if __name__ == "__main__":
    main()



Starting attendance system...
Loading known faces...
Finished loading known faces.
Starting thread for Camera 0...
Starting thread for Camera 1...
Camera 1 started.
Camera 0 started.
Detected srinath on Camera 0.
Entry logged for srinath.
Detected akhil on Camera 0.
Entry logged for akhil.
Detected srinath on Camera 0.
Detected Unknown on Camera 0.
Entry logged for Unknown.
Detected srinath on Camera 0.
Detected akhil on Camera 0.
Detected Unknown on Camera 1.
Detected Unknown on Camera 1.
Detected akhil on Camera 0.
Detected Unknown on Camera 0.
Detected Unknown on Camera 1.
Detected Unknown on Camera 1.
Detected Unknown on Camera 0.
Auto exit logged for srinath.
Detected shreyesh on Camera 1.
Exit logged for shreyesh.


In [6]:

import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
import csv
from datetime import datetime, timedelta
import threading
with open(r'C:\attendance2\attendance_log.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Test', datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
print("Write test successful")



Write test successful


In [ ]:
import cv2
import numpy as np
from retinaface import RetinaFace
import face_recognition
import os
from datetime import datetime
import threading
import openpyxl

# Directories and Constants
KNOWN_FACES_DIR = r"C:\attendance2\images"
EXCEL_FILE_PATH = r"C:\attendance2\Book1.xlsx"
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2

# Load known faces
def load_known_faces():
    known_faces = []
    known_names = []

    print("Loading known faces...")
    for name in os.listdir(KNOWN_FACES_DIR):
        person_dir = os.path.join(KNOWN_FACES_DIR, name)
        for filename in os.listdir(person_dir):
            filepath = os.path.join(person_dir, filename)
            image = face_recognition.load_image_file(filepath)
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_faces.append(encodings[0])
                known_names.append(name)
            else:
                print(f"Warning: No face found in {filename}")
    print("Finished loading known faces.")
    return known_faces, known_names

# Update Excel function
def update_excel(name, subject, in_time, out_time):
    wb = openpyxl.load_workbook(EXCEL_FILE_PATH)
    sheet = wb.active

    # Find the student row by matching the name in column A
    student_row = None
    for row in range(2, sheet.max_row + 1):
        if sheet.cell(row, 1).value == name:
            student_row = row
            break

    if student_row is None:
        print(f"Error: Student {name} not found in Excel.")
        return

    # Calculate total time in minutes
    total_time = (out_time - in_time).total_seconds() / 60

    # Update the subject's data in the Excel
    subject_col = 2 + (subject - 1) * 5  # Each subject takes 5 columns
    print(f"Updating {name} for subject {subject}: IN: {in_time}, OUT: {out_time}, TOTAL TIME: {total_time:.2f} mins")

    sheet.cell(student_row, subject_col).value = in_time.strftime('%H:%M:%S')
    sheet.cell(student_row, subject_col + 1).value = out_time.strftime('%H:%M:%S')
    sheet.cell(student_row, subject_col + 2).value = f"{total_time:.2f} mins"

    # Increment periods attended if total time > 45 minutes
    periods_attended = sheet.cell(student_row, subject_col + 3).value or 0
    if total_time > 45:
        periods_attended += 1
    sheet.cell(student_row, subject_col + 3).value = periods_attended

    # Calculate daily attendance percentage
    total_periods_attended = sum(sheet.cell(student_row, 5 + i * 5).value or 0 for i in range(6))
    daily_percentage = (total_periods_attended / 6) * 100
    sheet.cell(student_row, sheet.max_column).value = f"{daily_percentage:.2f}%"

    # Save the Excel file with updates
    try:
        wb.save(EXCEL_FILE_PATH)
        print(f"Updated attendance for {name} in subject {subject}.")
    except Exception as e:
        print(f"Error saving Excel file: {e}")

# Drawing faces function
def draw_faces(frame, faces, names):
    for (face, name) in zip(faces, names):
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), FRAME_THICKNESS)
        y_position = face[1] - 15 if face[1] - 15 > 15 else face[1] + 15
        cv2.rectangle(frame, (face[0], face[1]), (face[2], face[1] - 35), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (face[0] + 6, y_position), font, 0.8, (255, 255, 255), FONT_THICKNESS)

# Recognizing faces from camera
def recognize_faces_from_camera(camera_id, known_faces, known_names, tracking_dict):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Camera {camera_id} failed to open.")
        return

    print(f"Camera {camera_id} started.")
    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print(f"Failed to read frame from Camera {camera_id}.")
            break

        faces = RetinaFace.detect_faces(frame)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = []
        face_names = []

        for key in faces.keys():
            facial_area = faces[key]["facial_area"]
            face_locations.append(facial_area)

            top, right, bottom, left = facial_area[1], facial_area[2], facial_area[3], facial_area[0]
            encodings = face_recognition.face_encodings(frame_rgb, [(top, right, bottom, left)])

            for encoding in encodings:
                matches = face_recognition.compare_faces(known_faces, encoding, TOLERANCE)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_faces, encoding)
                best_match_index = np.argmin(face_distances)

                if matches[best_match_index]:
                    name = known_names[best_match_index]
                face_names.append(name)

                print(f"Detected {name} on Camera {camera_id}.")
                if camera_id == 0:  # Camera for IN
                    if name not in tracking_dict['in_time']:
                        tracking_dict['in_time'][name] = datetime.now()
                        print(f"{name} IN at {tracking_dict['in_time'][name]}")
                elif camera_id == 1:  # Camera for OUT
                    in_time = tracking_dict['in_time'].pop(name, None)
                    if in_time:
                        out_time = datetime.now()
                        update_excel(name, subject=1, in_time=in_time, out_time=out_time)
                        print(f"{name} OUT at {out_time}")

        draw_faces(frame, face_locations, face_names)
        cv2.imshow(f'Camera {camera_id}', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print(f"Camera {camera_id} stopped.")
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function
def main():
    print("Starting attendance system...")
    known_faces, known_names = load_known_faces()

    tracking_dict = {'in_time': {}}

    camera_ids = [0, 1]
    threads = []

    for camera_id in camera_ids:
        print(f"Starting thread for Camera {camera_id}...")
        thread = threading.Thread(target=recognize_faces_from_camera, args=(camera_id, known_faces, known_names, tracking_dict))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    print("Attendance system stopped.")

if __name__ == "__main__":
    main()



Starting attendance system...
Loading known faces...
Finished loading known faces.
Starting thread for Camera 0...
Starting thread for Camera 1...
Camera 1 started.
Camera 0 started.
Detected Unknown on Camera 0.
Unknown IN at 2024-10-16 15:40:44.892227
Detected Unknown on Camera 0.
Detected akhil on Camera 0.
akhil IN at 2024-10-16 15:40:57.790216
Detected Unknown on Camera 0.
Detected akhil on Camera 0.
Detected Unknown on Camera 0.
Detected shreyesh on Camera 0.
shreyesh IN at 2024-10-16 15:41:26.619832
Detected Unknown on Camera 0.
Detected srinath on Camera 0.
srinath IN at 2024-10-16 15:41:40.016124
Detected shreyesh on Camera 0.
Detected srinath on Camera 0.
Detected Unknown on Camera 0.


In [2]:
%pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.9 kB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 92.2/250.9 kB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 143.4/250.9 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
